In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [27]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    pmat = np.zeros([env.nS, env.nS, env.nA])
    for s in range(env.nS):
        for a in range(env.nA):
            for sprime in range(env.nS):
                alwaysOne, nextState, reward, done = env.P[s][a][0]
                pmat[sprime, s, a] = 1 if sprime == nextState else 0
    """  for sprime in range(env.nS):
                    alwaysOne, nextState, reward, done = env.P[state][action][0]
                    V[state] += policy[state,action]*pmat[sprime, state, action]*(reward + discount_factor*V[sprime])"""
    V = np.zeros(env.nS)
    while True:
        delta = 0
        for state in range(env.nS):
            v = V[state]
            placeholder = 0
            for action in range(env.nA):
                alwaysOne, nextState, reward, done = env.P[state][action][0]
                placeholder += policy[state, action]*(reward + discount_factor*V[nextState])
            
            V[state] = placeholder
            delta = max(delta, abs(V[state]-v))
        if delta < theta:
            break
    return np.array(V)

In [28]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [29]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)